In [282]:
import torch
from torch import nn
from torch.nn import functional
import numpy as np
import math

In [283]:
# Reading Tiny Shakespeare Dataset
file = open('tiny-shakespeare.txt', 'r')
text = file.read()
file.close()

In [405]:
head_size = 4
embedding_size = 5
class Single_Attention_Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin_q = nn.Linear(embedding_size, head_size, bias = False)
        self.lin_k = nn.Linear(embedding_size, head_size, bias = False)
        self.lin_v_up = nn.Linear(embedding_size, head_size, bias = False)
        self.lin_v_down = nn.Linear(head_size, head_size, bias = False)
        self.soft = nn.Softmax(dim = 2)

    def forward(self, embedding):
        Q = self.lin_q(embedding)
        Q = Q[:, :, None]
        #print(Q.shape)
        K = self.lin_k(embedding)
        K = K[:, :, None]
        K = torch.permute(K, (0, 2, 1))
        #print(K.shape)
        V = self.lin_v_up(embedding)
        V = self.lin_v_down(V)
        V = V[:, :, None]
        #print("V Shape : ", V.shape)
        attention_matrix = (Q @ K) / (math.sqrt(head_size))
        attention_matrix = torch.tril(attention_matrix)
        attention_matrix[attention_matrix == 0] = -math.inf
        attention_matrix = self.soft(attention_matrix)
        attention_matrix = attention_matrix @ V
        return attention_matrix

In [406]:
test_embedding = torch.rand(3, embedding_size)
test_embedding.shape

torch.Size([3, 5])

In [407]:
model = Single_Attention_Block()

In [408]:
result = model(test_embedding)

In [409]:
result

tensor([[[ 0.0665],
         [ 0.0718],
         [ 0.0666],
         [ 0.0453]],

        [[ 0.0550],
         [ 0.0894],
         [ 0.1086],
         [ 0.0699]],

        [[-0.1050],
         [ 0.0068],
         [ 0.0317],
         [ 0.0023]]], grad_fn=<UnsafeViewBackward0>)